## Cross Id Lookup Script
This script was used to extract ra and dec information from the Pan-STARRS galaxies contained in the [spin parity catalog](https://github.com/cora-schallock/spin-parity-catalog) in order to perform a cross id object lookup in the [SDSS DR12 sky survery](https://skyserver.sdss.org/dr12/en/tools/crossid/crossid.aspx)

In [28]:
import os
import csv

path_to_galaxies = "..\..\spin-parity-catalog\original\galaxies" #update this to be the path of the spin_parity_catalog/original/galaxies on your machine
tables_to_cross_match = ['table2', 'table3', 'table4', 'table5']

header = ["name","ra","dec"]
output_dir = "..\..\..\cross_id"


def standardize_gal_name(gal_name):
    return gal_name.strip().replace(" ","_")

def can_float(to_float):
    try:
        float(to_float)
        return True
    except:
        return False
    
def write_csv(csv_path,csv_col,csv_rows):
    to_write = [csv_col]
    to_write.extend(csv_rows)

    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(to_write) 

def get_ra_dec_file_path(name, table_name):
    return os.path.join(path_to_galaxies,table_name,name,"{}_ra_dec.txt".format(name))

def extract_ra_dec_from_file(file_path):
    with open(file_path,'r') as f:
        to_parse = f.read().replace("="," ").replace("("," ").replace(")"," ").replace(","," ").split(" ")
        if len(to_parse) < 10: return None

        ra = to_parse[5]; dec = to_parse[10]
        if not can_float(ra) or not can_float(dec): return None
        
        return (float(ra),float(dec))
    
def run_on_table(table_name):
    rows = []
    for gal_name in os.listdir(os.path.join(path_to_galaxies,table_name)):
        the_path = get_ra_dec_file_path(gal_name, table_name)
        if not os.path.exists(the_path): continue

        the_ra_and_dec = extract_ra_dec_from_file(the_path)
        if not isinstance(the_ra_and_dec,tuple) or not len(the_ra_and_dec) == 2: continue
        rows.append([standardize_gal_name(gal_name),the_ra_and_dec[0],the_ra_and_dec[1]])
    write_csv(os.path.join(output_dir,"{}.csv".format(table_name)),header,rows)

run_on_table('table5')